In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    print(Model)
    purge(Model)
db.session.commit()

<class 'app.models.Bracket'>
<class 'app.models.Tournament'>
<class 'app.models.Match'>
<class 'app.models.Round'>
<class 'app.models.User'>


In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df

,id,role,username,email,password_hash,about_me,last_seen


In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)

# print empty user table
df


,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$3YTworc0$9b450d0f0f7d6858...,None,2020-03-23 22:35:20.528096


In [10]:
r.headers['location']

'http://127.0.0.1:5000/api/user/1'

In [11]:
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })

    r = requests.post(url, data=payload, headers=headers)
#     print(vars(r))

In [12]:
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$3YTworc0$9b450d0f0f7d6858...,None,2020-03-23 22:35:20.528096
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$4wzxlnD4$bc05cb6444d54aac...,None,2020-03-23 22:35:20.953799
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$RkGYtixR$ef7033e46a27bbc4...,None,2020-03-23 22:35:21.248119
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$AS8nVKYd$9fa4088f91634e50...,None,2020-03-23 22:35:21.401728
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$GJyRhpWx$95833b39241e3931...,None,2020-03-23 22:35:21.499937
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$P3pOfCWo$785c09a1fb600034...,None,2020-03-23 22:35:21.583449
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$kb4t9pal$383ad35d2d64acbe...,None,2020-03-23 22:35:21.669998
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$hRxyvTAW$96808b5b78bf1195...,None,2020-03-23 22:35:21.771172
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$Do1HW6Oi$3fa44281753d6f31...,None,2020-03-23 22:35:21.887128


In [13]:
q = """
select * from lobby_seeds
"""
# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,user_id,lobby_id,seed


In [14]:
# whenever a user registers, the response contains the url to that user's homepage
r.headers['location']

'http://127.0.0.1:5000/api/user/9'

In [15]:
# login with miguel credentials
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/login

url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=('miguel', 'python'))

In [16]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [17]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMjkyMiwiZXhwIjoxNTg1MDA2NTIyfQ.eyJpZCI6MX0.QV3SHMVwHzx0I-wlHAyIkuBOZv7YyGfIgJGmUEHZP0-rHs4xgMK-5y8tFLh7kBlycQybbO5XYw9azre04gJ1wg'

In [18]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [19]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourney"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content

b'{"Unique Constraint Failed":"User \\"1\\" already created tournament \\"miguel\'s tourney\\""}\n'

In [20]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourney,None,1


In [21]:
# now this user will create a tournament

In [22]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [23]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [24]:
# this our bracket (input to bracket creation):
tuple_list

[('TPN', 5),
 ('AngryFalco', 1),
 ('Kevin', 4),
 ('Burnaby', 2),
 ('Spaceghost', 8),
 ('Ptolemy', 6),
 ('Vik', 7),
 ('Sunrisebanana', 3)]

In [25]:
# create a json payload from the tuple list
url = 'http://127.0.0.1:5000/api/created-tournaments/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "users":usernames,
    "seeds":seeds,
    "tournament_name":"Fall Charity LAN 2018 Melee Singles"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r

<Response [200]>

In [26]:
q = \
"""
    select * from match where user_1 and user_2
"""

In [27]:
df = pd.read_sql_query(q, cnx)
df

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,cc71d1e6-4af4-4e9c-a730-188d13c207dd,3,None,8,None,None,1,5,10
1,2,acb7d3ba-b2e8-4888-b1e6-91c8d3755b9e,7,None,2,None,None,1,5,10
2,3,ce5194fa-d616-48de-beb6-8a5e30e10072,9,None,6,None,None,1,6,11
3,4,788bf752-be66-4abe-a9a5-538e0901272c,4,None,5,None,None,1,6,11


In [28]:
active_match_ids = df.id.values
active_match_ids

array([1, 2, 3, 4])

In [29]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMjkyMiwiZXhwIjoxNTg1MDA2NTIyfQ.eyJpZCI6MX0.QV3SHMVwHzx0I-wlHAyIkuBOZv7YyGfIgJGmUEHZP0-rHs4xgMK-5y8tFLh7kBlycQybbO5XYw9azre04gJ1wg'

In [30]:
r = requests.get(url, auth=('TPN', 'tpn'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMjkyMiwiZXhwIjoxNTg1MDA2NTIyfQ.eyJpZCI6Mn0.K_5_PznfPROWtLm_4Ynp1DkaUUgjT5q8Toy0dksAQ5BIRQrsg6bopk1Rnl6yK-tO-6UO-k_2T85rPKOpz8s2qg'

In [31]:
payload = json.dumps({
        "entrant1_score" : 3,
        "entrant2_score" : 2,
        "winner" : 9,
        "loser" : 8
})

In [32]:
url = f'http://127.0.0.1:5000/api/match/1/report-match'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content

b'{"Access Denied":"Only Tournament Organizers can report matches"}\n'

In [33]:
# for i in active_match_ids:
#     user_ids = [
#         df.loc[df['id'] == i].user_1.values[0],
#         df.loc[df['id'] == i].user_2.values[0]
#     ]
#     scores = random.sample(range(5), 2)
#     w_id = user_ids[np.argmax(scores)]
#     l_id = user_ids[np.argmin(scores)]
# #     print(w_id)
# #     print(l_id)
#     payload = json.dumps({
#         "entrant1_score" : str(scores[0]),
#         "entrant2_score" : str(scores[1]),
#         "winner" : str(w_id),
#         "loser" : str(l_id)
#     })
#     url = f'http://127.0.0.1:5000/api/match/{i}/report-match'
#     headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
#     r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
#     print(r.content)
#     break
    

In [34]:
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "entrant1_score":4,
#     "entrant2_score":3,
#     "winner":,
#     "loser":,
# })

In [35]:

# TO = t.bracket.entrants[0]

# # initial post to db
# t.post_to_db(tournament_name, TO,)

In [36]:
# # post self references in matches separately
# for r in t.bracket.rounds:
#     for m in r.matches:
#         m.post_self_refs()

In [37]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$3YTworc0$9b450d0f0f7d6858...,None,2020-03-23 22:35:20.528096
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$4wzxlnD4$bc05cb6444d54aac...,None,2020-03-23 22:35:20.953799
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$RkGYtixR$ef7033e46a27bbc4...,None,2020-03-23 22:35:21.248119
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$AS8nVKYd$9fa4088f91634e50...,None,2020-03-23 22:35:21.401728
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$GJyRhpWx$95833b39241e3931...,None,2020-03-23 22:35:21.499937
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$P3pOfCWo$785c09a1fb600034...,None,2020-03-23 22:35:21.583449
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$kb4t9pal$383ad35d2d64acbe...,None,2020-03-23 22:35:21.669998
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$hRxyvTAW$96808b5b78bf1195...,None,2020-03-23 22:35:21.771172
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$Do1HW6Oi$3fa44281753d6f31...,None,2020-03-23 22:35:21.887128


In [38]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id,finished
0,1,8,Fall Charity LAN 2018 Melee Singles,1,0


In [39]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [40]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,3,1
1,9,1
2,4,1
3,7,1
4,2,1
5,5,1
6,6,1
7,8,1


In [41]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [42]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,cc71d1e6-4af4-4e9c-a730-188d13c207dd,3.0,NaN,8.0,NaN,None,1,5.0,10.0
1,2,acb7d3ba-b2e8-4888-b1e6-91c8d3755b9e,7.0,NaN,2.0,NaN,None,1,5.0,10.0
2,3,ce5194fa-d616-48de-beb6-8a5e30e10072,9.0,NaN,6.0,NaN,None,1,6.0,11.0
3,4,788bf752-be66-4abe-a9a5-538e0901272c,4.0,NaN,5.0,NaN,None,1,6.0,11.0
4,5,24cd436c-3e81-45c0-af0b-edab581e50f0,NaN,NaN,NaN,NaN,None,2,7.0,12.0
5,6,468424c3-a137-47ac-b1db-3c58ceb8cf00,NaN,NaN,NaN,NaN,None,2,7.0,13.0
6,7,1542054c-18f7-46f6-8401-7747617beb05,NaN,NaN,NaN,NaN,None,3,8.0,15.0
7,8,a51582ee-6503-4202-93c7-40d9ee53383c,NaN,NaN,NaN,NaN,None,4,9.0,9.0
8,9,4d270c78-09cb-4f92-9ea0-2a21494c03b8,NaN,NaN,NaN,NaN,None,5,NaN,NaN
9,10,bf17e541-081d-41cc-96da-6780a66a4e03,NaN,-1.0,NaN,0.0,None,6,12.0,NaN


In [43]:
url = 'http://127.0.0.1:5000/api/user/1'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({})
r = requests.get(url, data=payload, headers=headers, auth=(tkn, 'unused'))
r.content
json.loads(r.content)

{'about_me': None,
 'bracket_users': [],
 'brackets': [],
 'email': 'miguel@gmail.com',
 'id': 1,
 'last_seen': '2020-03-23T22:35:20.528096',
 'lobby_seeds': [],
 'password_hash': 'pbkdf2:sha256:150000$3YTworc0$9b450d0f0f7d6858c7dff1b3dbf02cb8e5942dc8496ec24d0f9b397bc7edfc0a',
 'role': 'User',
 'username': 'miguel'}

In [44]:
# tpn tries to modify miguel

In [45]:
url = 'http://127.0.0.1:5000/api/token'

r = requests.get(url, auth=('TPN', 'tpn'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMjkyMywiZXhwIjoxNTg1MDA2NTIzfQ.eyJpZCI6Mn0.U48acbuzU1s6Fp-1-tdN4Ay_qC5XWxXZWDw9kUcn5aYgzZKW7GpYVu_Js2kuwYOBfxjRQUL-1Nqv4Vw18kmN3Q'

In [46]:
url = 'http://127.0.0.1:5000/api/user/1/modify/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    'username' : 'miguel_is_stupid',
    'about_me' : 'he sucks'
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Unauthorized': 'TPN is not authorized to modify this user'}

In [47]:
# log miguel back in

In [48]:
url = 'http://127.0.0.1:5000/api/token'

r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4NTAwMjkyMywiZXhwIjoxNTg1MDA2NTIzfQ.eyJpZCI6MX0.BRALS-dEt0rb2wCGA36eGYYToKzct3WRtf-_ptqkIvJUfvpAr2E2TZPLBugpcupYnHnq1pyaEEtub_fQmYvmCg'

In [49]:
url = 'http://127.0.0.1:5000/api/user/1/modify/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    'username' : 'miguel_is_great',
    'about_me' : 'he\'s awesome'
}) 
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Success': 'Updated username and about_me'}

In [50]:
vars(r)

{'_content': b'{"Success":"Updated username and about_me"}\n',
 '_content_consumed': True,
 '_next': None,
 'status_code': 202,
 'headers': {'Content-Type': 'application/json', 'Content-Length': '44', 'Server': 'Werkzeug/1.0.0 Python/3.6.9', 'Date': 'Mon, 23 Mar 2020 22:35:23 GMT'},
 'raw': <urllib3.response.HTTPResponse at 0x7f4700c46dd8>,
 'url': 'http://127.0.0.1:5000/api/user/1/modify/',
 'encoding': None,
 'history': [],
 'reason': 'ACCEPTED',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 0, 16134),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7f4700c46780>}

In [51]:
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel_is_great,miguel@gmail.com,pbkdf2:sha256:150000$3YTworc0$9b450d0f0f7d6858...,he's awesome,2020-03-23 22:35:20.528096
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$4wzxlnD4$bc05cb6444d54aac...,None,2020-03-23 22:35:20.953799
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$RkGYtixR$ef7033e46a27bbc4...,None,2020-03-23 22:35:21.248119
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$AS8nVKYd$9fa4088f91634e50...,None,2020-03-23 22:35:21.401728
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$GJyRhpWx$95833b39241e3931...,None,2020-03-23 22:35:21.499937
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$P3pOfCWo$785c09a1fb600034...,None,2020-03-23 22:35:21.583449
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$kb4t9pal$383ad35d2d64acbe...,None,2020-03-23 22:35:21.669998
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$hRxyvTAW$96808b5b78bf1195...,None,2020-03-23 22:35:21.771172
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$Do1HW6Oi$3fa44281753d6f31...,None,2020-03-23 22:35:21.887128


In [52]:
# try to steal jaime's name

url = 'http://127.0.0.1:5000/api/user/1/modify/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    'username' : 'Sunrisebanana',
#     'about_me' : 'he\'s awesome'
}) 
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Invalid Username': 'Sunrisebanana Already Taken'}

In [ ]:
# plagiarize jaime's name

url = 'http://127.0.0.1:5000/api/user/1/modify/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    'username' : 'Sunrisebanana',
#     'about_me' : 'he\'s awesome'
}) 
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)